You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Filter Operation:
o	Perform a "filter" operation on the retrieved launch data based on specific criteria.
o	Filter Logic: Filter the list of launches based on launch year and launch success status. Only include launches that match the specified criteria.
o	The result should be a list of dictionaries, containing only the launch records that satisfy the filter conditions.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URL and the filtering criteria, making them easily modifiable. Use parameters like: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
	FILTER_YEAR = 2019 (Set to an integer year, or None to disable year filtering)
	FILTER_SUCCESS = True (Set to True, False, or None to disable success filtering)
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, filtering criteria being applied, number of launches before filtering, number of launches after filtering, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + filtering). Print this time after the filtering operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting filtered list from the filter operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting filtered JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
import requests
import logging
import time
import json
from typing import List, Dict, Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Control parameters - easily modifiable
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
FILTER_YEAR = 2019  # Set to integer year or None to disable year filtering
FILTER_SUCCESS = True  # Set to True, False, or None to disable success filtering
HTTPBIN_UPLOAD_URL = "https://httpbin.org/post"

def retrieve_launch_data(api_url: str) -> List[Dict]:
    """
    Retrieve launch data from SpaceX API.
    
    Args:
        api_url: URL of the SpaceX API endpoint
        
    Returns:
        List of launch records as dictionaries
        
    Raises:
        requests.exceptions.RequestException: If API request fails
    """
    logger.info(f"Starting data retrieval from {api_url}")
    
    try:
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()  # Raises exception for 4XX/5XX status codes
        
        data = response.json()
        logger.info(f"Successfully retrieved {len(data)} launches")
        return data
        
    except requests.exceptions.Timeout:
        logger.error("Request timed out while retrieving launch data")
        raise
    except requests.exceptions.HTTPError as e:
        logger.error(f"HTTP error occurred: {e.response.status_code} - {e.response.text}")
        raise
    except requests.exceptions.RequestException as e:
        logger.error(f"Error occurred during API request: {e}")
        raise

def filter_launches(launches: List[Dict], 
                   year: Optional[int] = None, 
                   success: Optional[bool] = None) -> List[Dict]:
    """
    Filter launch data based on year and success criteria.
    
    Args:
        launches: List of launch records
        year: Year to filter by (None to disable)
        success: Success status to filter by (None to disable)
        
    Returns:
        Filtered list of launch records
    """
    logger.info(f"Starting filtering - {len(launches)} launches before filtering")
    
    filtered_data = launches.copy()
    
    # Apply filters if criteria are specified
    if year is not None:
        logger.info(f"Applying year filter: {year}")
        filtered_data = [launch for launch in filtered_data 
                        if launch.get('launch_year') == str(year)]
    
    if success is not None:
        logger.info(f"Applying success filter: {success}")
        filtered_data = [launch for launch in filtered_data 
                        if launch.get('launch_success') == success]
    
    logger.info(f"Filtering complete - {len(filtered_data)} launches after filtering")
    return filtered_data

def upload_to_httpbin(data: List[Dict], upload_url: str) -> bool:
    """
    Upload filtered data to httpbin.org
    
    Args:
        data: Data to upload
        upload_url: httpbin endpoint URL
        
    Returns:
        bool: True if upload was successful, False otherwise
    """
    logger.info(f"Starting upload to {upload_url}")
    
    try:
        json_data = json.dumps(data)
        response = requests.post(upload_url, data=json_data, headers={'Content-Type': 'application/json'})
        response.raise_for_status()
        
        logger.info(f"Upload successful - Status code: {response.status_code}")
        logger.debug(f"Response from httpbin: {response.json()}")
        return True
        
    except requests.exceptions.RequestException as e:
        logger.error(f"Error occurred during upload: {e}")
        return False

def main():
    """Main execution pipeline"""
    # Start total execution timer
    total_start_time = time.time()
    
    try:
        # Measure data retrieval and filtering time
        start_time = time.time()
        
        # Step 1: Retrieve launch data
        launches = retrieve_launch_data(API_ENDPOINT_URL)
        
        # Step 2: Filter launches
        filtered_launches = filter_launches(launches, FILTER_YEAR, FILTER_SUCCESS)
        
        # Log execution time for data retrieval + filtering
        elapsed_time = time.time() - start_time
        logger.info(f"Data retrieval and filtering completed in {elapsed_time:.2f} seconds")
        
        # Step 3: Upload to httpbin
        upload_success = upload_to_httpbin(filtered_launches, HTTPBIN_UPLOAD_URL)
        
        # Log total execution time
        total_elapsed_time = time.time() - total_start_time
        logger.info(f"Total pipeline execution time: {total_elapsed_time:.2f} seconds")
        
        return upload_success
        
    except Exception as e:
        logger.error(f"Pipeline failed with error: {e}")
        return False

if __name__ == "__main__":
    main()